In [67]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import ast
import pandas as pd

# Grab the 'menu' item from the American Kennel Club (AKC) website
dog_breed_main = requests.get('https://www.akc.org/dog-breeds/') 
soup = BeautifulSoup(dog_breed_main.content, 'html.parser')
menu = soup.find('div', {'class':'custom-select'})

In [68]:
# Then re-parse menu to get list of links to each breed
menu_soup = BeautifulSoup(str(menu), 'html.parser')
breed_results = menu_soup.find_all('option')[1:]
breed_links = [result['value'] for result in breed_results]

In [69]:
# Scrape each link for information

prefix = 'googletag.pubads().setTargeting'
dog_char_list = []

for link in tqdm(breed_links):
    breed_html = requests.get(link)
    bs = BeautifulSoup(breed_html.content, 'html.parser')
    js_ads = [script for script in bs.find_all('script') if "googletag.pubads()" in str(script)][0]
    pub_ads = [ad.strip()[len(prefix):-1] for ad in str(js_ads).split('\n') if 'googletag.pubads().setTargeting' in ad]
    dog_chars = [ast.literal_eval(char) for char in pub_ads] 
    dog_char_list.append(dog_chars)

100%|██████████| 281/281 [01:54<00:00,  2.45it/s]


In [70]:
print(len(dog_char_list)) 

281
